In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# sound
import winsound
import time
import datetime

In [22]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)


device:  cuda


In [23]:
# Hyper-parameters 
batch_size = 4
# dataset has PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
#imshow(torchvision.utils.make_grid(images))

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x


Files already downloaded and verified
Files already downloaded and verified


In [24]:
# Hyper-parameters 
num_epochs = 10
learning_rate = 1e-3


model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
#PATH = './cnn.pth'
#torch.save(model.state_dict(), PATH)

Epoch [1/10], Step [2000/12500], Loss: 2.3154
Epoch [1/10], Step [4000/12500], Loss: 2.3024
Epoch [1/10], Step [6000/12500], Loss: 2.2876
Epoch [1/10], Step [8000/12500], Loss: 2.2846
Epoch [1/10], Step [10000/12500], Loss: 2.1066
Epoch [1/10], Step [12000/12500], Loss: 1.9891
Epoch [2/10], Step [2000/12500], Loss: 1.7724
Epoch [2/10], Step [4000/12500], Loss: 1.4838
Epoch [2/10], Step [6000/12500], Loss: 1.7266
Epoch [2/10], Step [8000/12500], Loss: 1.7164
Epoch [2/10], Step [10000/12500], Loss: 1.8042
Epoch [2/10], Step [12000/12500], Loss: 2.0882
Epoch [3/10], Step [2000/12500], Loss: 1.7690
Epoch [3/10], Step [4000/12500], Loss: 1.1664
Epoch [3/10], Step [6000/12500], Loss: 2.3090
Epoch [3/10], Step [8000/12500], Loss: 1.0515
Epoch [3/10], Step [10000/12500], Loss: 1.4035
Epoch [3/10], Step [12000/12500], Loss: 1.0886
Epoch [4/10], Step [2000/12500], Loss: 1.2758
Epoch [4/10], Step [4000/12500], Loss: 2.1923
Epoch [4/10], Step [6000/12500], Loss: 1.2154
Epoch [4/10], Step [8000/125

In [25]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 56.78 %
Accuracy of plane: 57.8 %
Accuracy of car: 74.7 %
Accuracy of bird: 56.3 %
Accuracy of cat: 40.8 %
Accuracy of deer: 41.9 %
Accuracy of dog: 39.1 %
Accuracy of frog: 69.6 %
Accuracy of horse: 62.6 %
Accuracy of ship: 69.5 %
Accuracy of truck: 55.5 %
